# Graph Visualization

<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/notebooks/networkViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>

## requirements.txt


In [ ]:
!pip install -q minio
!pip install -q kglab
!pip install -q sparqlwrapper
!pip install -q pygraphml
!pip install -q ipysigma

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import kglab
from minio import Minio
from rdflib import Graph, plugin
import plotly.express as px
import pandas as pd
from urllib.request import urlopen
import os,json

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
## Remote SPARQL server
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



In [3]:
rp1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType
WHERE {
  graph ?g {
    ?source a ?sType .
    ?target a ?tType .
    ?source ?type ?target .
    FILTER((?sType) IN (schema:Person, schema:Organization, schema:Dataset, schema:Course, schema:Document))
    FILTER((?tType) IN (schema:Person, schema:Organization, schema:Dataset, schema:Course, schema:Document))
  }
    FILTER (!REGEX(STR(?g), "bcodmo"))

}

"""

ep = "http://0.0.0.0:7878/query"

df = get_sparql_dataframe(ep, rp1)


In [4]:
df.head(10)


,source,type,target,sType,tType
0,https://obis.org/dataset/35abcc7d-09c6-4189-98...,https://schema.org/provider,https://oceanexpert.org/institution/19457,https://schema.org/Dataset,https://schema.org/Organization
1,https://obis.org/dataset/35abcc7d-09c6-4189-98...,https://schema.org/provider,https://oceanexpert.org/institution/18705,https://schema.org/Dataset,https://schema.org/Organization
2,https://obis.org/dataset/35abcc7d-09c6-4189-98...,https://schema.org/provider,https://oceanexpert.org/institution/5163,https://schema.org/Dataset,https://schema.org/Organization
3,https://obis.org/dataset/5ec11ed9-4b39-4078-a7...,https://schema.org/provider,https://oceanexpert.org/institution/19299,https://schema.org/Dataset,https://schema.org/Organization
4,https://obis.org/dataset/f5eb3f1e-d0b9-4878-90...,https://schema.org/provider,https://oceanexpert.org/institution/19393,https://schema.org/Dataset,https://schema.org/Organization
5,https://obis.org/dataset/afb23951-7a91-44f5-9c...,https://schema.org/provider,https://oceanexpert.org/institution/20836,https://schema.org/Dataset,https://schema.org/Organization
6,https://obis.org/dataset/afb23951-7a91-44f5-9c...,https://schema.org/provider,https://oceanexpert.org/institution/23181,https://schema.org/Dataset,https://schema.org/Organization
7,https://obis.org/dataset/afb23951-7a91-44f5-9c...,https://schema.org/provider,https://oceanexpert.org/institution/19393,https://schema.org/Dataset,https://schema.org/Organization
8,https://obis.org/dataset/d6229742-b1bc-4db8-bb...,https://schema.org/provider,https://oceanexpert.org/institution/18704,https://schema.org/Dataset,https://schema.org/Organization
9,https://obis.org/dataset/1e69cd84-1703-4383-ae...,https://schema.org/provider,https://oceanexpert.org/institution/5028,https://schema.org/Dataset,https://schema.org/Organization


In [5]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!  
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])
g = GraphML()
g.directed = False

for index, row in df.iterrows():
    n1 = g.add_node(row['source'])
    n1['type'] = row['sType']
    n2 = g.add_node(row['target'])
    n2['type'] = row['tType']
    e = g.add_edge(n1, n2)
    e['predicate'] = row['type']

fname = "./output/testGraphML.xml"
parser = GraphMLParser()
parser.write(g, fname)

In [6]:
# g = nx.les_miserables_graph()
g = nx.read_graphml("./output/testGraphML.xml")

In [7]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=3
)



Sigma(nx.Graph with 5,678 nodes and 8,461 edges)